## Test Selenium

In [1]:
!pip install selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Edge()
driver.get("https://www.wowhead.com/sounds/name:WorgenFemale")

In [12]:
links = driver.find_elements(By.CSS_SELECTOR, '#lv-sounds table tr>td:first-child a')
playlist_urls = [link.get_attribute('href') for link in links]
print(len(playlist_urls))
playlist_urls[:10]

50


['https://www.wowhead.com/sound=19404/vo-pcworgenfemale-erritemlocked',
 'https://www.wowhead.com/sound=22452/worgenfemale-wound',
 'https://www.wowhead.com/sound=19400/vo-pcworgenfemale-errinventoryfull',
 'https://www.wowhead.com/sound=19423/vo-pcworgenfemale-errspellcooldown',
 'https://www.wowhead.com/sound=19441/vo-pcworgenfemale-silly',
 'https://www.wowhead.com/sound=22447/worgenfemale-battleroar',
 'https://www.wowhead.com/sound=19385/vo-pcworgenfemale-errammoonly',
 'https://www.wowhead.com/sound=19386/vo-pcworgenfemale-errbagfull',
 'https://www.wowhead.com/sound=19388/vo-pcworgenfemale-errcantdropsoulbounditem',
 'https://www.wowhead.com/sound=19389/vo-pcworgenfemale-errcantequipever']

In [13]:
next_btn = driver.find_element(By.CSS_SELECTOR, '#lv-sounds .listview-nav').find_element(By.PARTIAL_LINK_TEXT, "Next")
is_next_active = next_btn.get_attribute('data-active') == 'yes'
is_next_active

True

In [16]:
driver.execute_script("arguments[0].click();", next_btn)
is_next_active = next_btn.get_attribute('data-active') == 'yes'
is_next_active

False

In [19]:
driver.get(playlist_urls[1])

In [21]:
links = driver.find_elements(By.CSS_SELECTOR, '#sound-file-list a')
sound_urls = [link.get_attribute('href') for link in links]
print(len(sound_urls))
sound_urls

10


['https://wow.zamimg.com/sound-ids/live/enus/182/564406/WorgenFemale_Wound_01.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/216/564440/WorgenFemale_Wound_02.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/166/564390/WorgenFemale_Wound_03.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/204/564428/WorgenFemale_Wound_04.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/214/564438/WorgenFemale_Wound_05.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/155/564379/WorgenFemale_Wound_06.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/151/564375/WorgenFemale_Wound_07.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/167/564391/WorgenFemale_Wound_08.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/178/564402/WorgenFemale_Wound_09.ogg',
 'https://wow.zamimg.com/sound-ids/live/enus/174/564398/WorgenFemale_Wound_10.ogg']

In [22]:
!pip install aiohttp aiofile

     ---------------------------------------- 0.0/324.5 kB ? eta -:--:--
     ---------- ----------------------------- 81.9/324.5 kB 2.3 MB/s eta 0:00:01
     ------------------------------ --------- 245.8/324.5 kB 3.0 MB/s eta 0:00:01
     ---------------------------------------- 324.5/324.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/56.9 kB ? eta -:--:--
     ---------------------------------------- 56.9/56.9 kB ? eta 0:00:00
  Created wheel for aiofile: filename=aiofile-3.8.1-py3-none-any.whl size=19501 sha256=1bfb596484d0cb5283a6a78e1171a8500ef78d548b90e4be8a38cfd2798bcb1c
  Stored in directory: c:\users\iambl\appdata\local\pip\cache\wheels\1a\8a\cb\64f9ddff5de83a58542726a9058913d0df84507b6e47d1f593
Successfully built aiofile


In [26]:
import os
import asyncio
import aiohttp
import aiofile

OUT_PATH = "test_out"
os.makedirs(OUT_PATH, exist_ok=True)
sema = asyncio.BoundedSemaphore(10)

async def fetch_file(url):
    fname = url.split("/")[-1]
    async with sema, aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            assert resp.status == 200
            data = await resp.read()

    async with aiofile.async_open(
        os.path.join(OUT_PATH, fname), "wb"
    ) as outfile:
        await outfile.write(data)
        
await asyncio.gather(*[fetch_file(url) for url in sound_urls])

[None, None, None, None, None, None, None, None, None, None]

In [28]:
asyncio.get_event_loop()

<_WindowsSelectorEventLoop running=True closed=False debug=False>

In [27]:
driver.quit()

In [43]:
from threading import Thread

async def g(x):
    await asyncio.sleep(1)
    print(x)

def f(loop):
    asyncio.run_coroutine_threadsafe(g("Hello"), loop)

t = Thread(target=f, args=(loop,))
t.start() 

Hello


## Run Selenium Scraper

In [32]:
from importlib import reload
import pools.selenium
reload(pools.selenium)
from pools.selenium import DriverPool, WindowPool

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

OUT_PATH = "raw_data"
os.makedirs(OUT_PATH, exist_ok=True)
sema = asyncio.BoundedSemaphore(10)
loop = asyncio.get_event_loop()

async def fetch_file(url):
    fname = url.split("/")[-1]
    async with sema, aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            assert resp.status == 200
            data = await resp.read()

    async with aiofile.async_open(
        os.path.join(OUT_PATH, fname), "wb"
    ) as outfile:
        await outfile.write(data)

def parse_searchPage(driver, pool):
    next_btn = driver.find_element(By.CSS_SELECTOR, '#lv-sounds .listview-nav').find_element(By.PARTIAL_LINK_TEXT, "Next")
    
    while True:
        links = driver.find_elements(By.CSS_SELECTOR, '#lv-sounds table tr>td:first-child a')
        for link in links:
            pool.scrape(link.get_attribute('href'), parse_playlistPage, pool)
            
        is_next_active = next_btn.get_attribute('data-active') == 'yes'
        if not is_next_active:
            break
            
        driver.execute_script("arguments[0].click();", next_btn)
    
def parse_playlistPage(driver, pool):
    links = driver.find_elements(By.CSS_SELECTOR, '#sound-file-list a')
    sound_urls = [link.get_attribute('href') for link in links]
    tasks = [fetch_file(url) for url in sound_urls]
    for task in tasks:
        loop.call_soon_threadsafe(asyncio.async, task)

In [46]:
url = "https://www.wowhead.com/sounds/name:WorgenFemale"

with DriverPool(5, headless=True) as pool:
    pool.scrape(url, parse_searchPage, pool)

[22540] Initializing webdriver.
[22540] Webdriver initialized.
[22540] Loading URL: https://www.wowhead.com/sounds/name:WorgenFemale
[22540] Finished loading.
[5788] Initializing webdriver.
[38580] Initializing webdriver.
[36140] Initializing webdriver.
[16140] Initializing webdriver.
[22540] Loading URL: https://www.wowhead.com/sound=19404/vo-pcworgenfemale-erritemlocked
[22540] Finished loading.
[22540] Loading URL: https://www.wowhead.com/sound=22452/worgenfemale-wound
[22540] Finished loading.
[22540] Loading URL: https://www.wowhead.com/sound=19400/vo-pcworgenfemale-errinventoryfull
[5788] Webdriver initialized.[38580] Webdriver initialized.
[38580] Loading URL: https://www.wowhead.com/sound=19423/vo-pcworgenfemale-errspellcooldown

[5788] Loading URL: https://www.wowhead.com/sound=19441/vo-pcworgenfemale-silly
[16140] Webdriver initialized.
[16140] Loading URL: https://www.wowhead.com/sound=22447/worgenfemale-battleroar
[36140] Webdriver initialized.
[36140] Loading URL: https://

ValueError: list.remove(x): x not in list